<a href="https://colab.research.google.com/github/Vakhranev/MDB/blob/main/%D0%A2%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
import csv
import re
import pandas as pd
import numpy as np
import os
import zipfile

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# распаковываем архив
fantasy_zip = zipfile.ZipFile('Transcribed data txt Summer STANDARDIZED 2017 2018 2019 2020.zip')
fantasy_zip.extractall()
fantasy_zip.close()

In [4]:
path ="Transcribed data txt Summer STANDARDIZED 2017 2018 2019 2020"
filelist = []

for root, dirs, files in os.walk(path):
  for file in files:
    if str(file)[-3:]=='txt':
      filelist.append(os.path.join(root,file))

print(len(filelist))

1123


In [36]:
import re
import pandas as pd
import numpy as np
import nltk
import csv

# Создаем csv-файл и записываем в него заголовки столбцов
with open('mdb_files.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['File Name', 'Length', 'First Name', 'Last Name', 'Entry/Exit', 'Mark', 'Year'])

# Проходим по списку файлов
for name in filelist:
    # Вытаскиваем название файла из архива
    FileName = re.split(r'/', name)[-1]

    # Извлекаем год из имени файла (первые 4 цифры)
    Year = re.findall(r'\d{4}', FileName)[0] if re.findall(r'\d{4}', FileName) else ''

    # Удаляем год из имени файла
    Names_of_file = re.sub(r'\d', '', FileName)

    # Заменяем пробельные символы на нижние подчёркивания
    Names_of_file = re.sub(r'\s+(?!\.)', '_', Names_of_file)

    # Удаляем повторяющиеся подчёркивания
    Names_of_file = re.sub(r'(?<!_)_(?!_)', '_', Names_of_file)

    # Заменяем последовательности подчёркиваний на одно подчёркивание
    Names_of_file = re.sub(r'_{2,}', '_', Names_of_file)

    # Удаляем подчёркивания в начале и конце строки
    Names_of_file = re.sub(r'^_+|_+$', '', Names_of_file)

    # Экранируем специальные символы в строке перед использованием в регулярном выражении
    Names_of_file_escaped = re.escape(Names_of_file)

    # Разбиваем названия файла по нижним подчёркиваниям и удаляем последний элемент списка
    List_with_names = Names_of_file_escaped.split("_")[:-1]

    # Проверяем, если последний элемент List_with_names равен 'ST', удаляем его
    if List_with_names[-1] == 'ST':
        List_with_names.pop()

    # Делаем строчный регистр и группируем имена правильно, если имен больше одного
    FN = List_with_names[0].lower()
    if len(List_with_names) > 3:
        List_with_names[0] = List_with_names[0] + " " + List_with_names[1]
        List_with_names[1] = List_with_names[2]
        List_with_names[2] = List_with_names[3]

    List_with_names[0] = List_with_names[0].lower()
    List_with_names[1] = List_with_names[1].lower()

    # Считываем данные из файла metadata.csv
    df = pd.read_csv('metadata.csv', sep=',')
    df.index = np.arange(1, len(df) + 1)

    # Приводим имена из файла к строчному регистру
    df['Last_Name'] = df['Last_Name'].str.lower()
    df['First_Name'] = df['First_Name'].str.lower()

    # Проверяем, совпадают ли имена в названиях файла с именами таблицы
    df1 = df[df["First_Name"].str.contains(FN)]
    df1 = df1[df1["Last_Name"].str.contains(List_with_names[1])]

    if df1.empty:
        df1 = df[df["Last_Name"].str.contains(FN)]
        df1 = df1[df1["First_Name"].str.contains(List_with_names[1])]
    if df1.empty:
        df1 = df.query("First_Name == @List_with_names[0]")
        df1 = df1.query("Last_Name == @List_with_names[1]")
    if df1.empty:
        df1 = df.query("Last_Name == @List_with_names[0]")
        df1 = df1.query("First_Name == @List_with_names[1]")

    # Проверяем, не содержат ли имена в таблице нижние подчёркивания вместо каких-нибудь символов
    if df1.empty:
        df1 = df[df["Last_Name"].str.contains('_')]
        df1 = df1[df1["First_Name"].str.contains(List_with_names[0])]

    # Вытаскиваем оценки
    if List_with_names[2] == 'Entry':
        mark = str(df1['Entry'])
        mark = re.split(r'\s', mark)[-5]
    else:
        mark = str(df1['Exit'])
        mark = re.split(r'\s', mark)[-5]

    # Проверяем, совпадает ли год из имени файла с годом в поле Year в данных из файла metadata
    while not df1.empty and str(df1[' Year'].iloc[0]) != Year:
        df1 = df1.iloc[1:]  # Пропускаем первую строку и продолжаем поиск

    if not df1.empty and str(df1[' Year'].iloc[0]) == Year:
        # Загоняем все в новый файл
        with open(name, 'r') as file:
            lines_in_file = file.read()

            nltk_tokens = nltk.word_tokenize(lines_in_file)
            nltk_tokens = " ".join(word for word in nltk_tokens if word not in ('!', '.', ':', ',', '–', '?', '(', ')', ';', '`', '<', '>'))
            nltk_tokens = nltk.word_tokenize(nltk_tokens)
            with open('mdb_files.csv', 'a') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                filewriter.writerow([FileName, len(nltk_tokens), List_with_names[0], List_with_names[1], List_with_names[2], mark, Year])
    else:
        print(f"Для файла {FileName} не найдено совпадение года в данных из файла metadata.csv")

Для файла Katherine_Sandy_Exit2_ST.txt не найдено совпадение года в данных из файла metadata.csv
Для файла Cannon_Williams_Exit2_ST.txt не найдено совпадение года в данных из файла metadata.csv
Для файла Kyle_Galerston_Exit2_ST.txt не найдено совпадение года в данных из файла metadata.csv
Для файла Michael_Gasparrini_Exit2_ST.txt не найдено совпадение года в данных из файла metadata.csv
Для файла Keelyn_O'Brien_Exit2_ST.txt не найдено совпадение года в данных из файла metadata.csv
Для файла Dylan_Courville_Exit2_ST.txt не найдено совпадение года в данных из файла metadata.csv
Для файла Angelika_Toomey_Exit2_ST.txt не найдено совпадение года в данных из файла metadata.csv
Для файла Alison_DiMaio_Exit2_ST.txt не найдено совпадение года в данных из файла metadata.csv
Для файла Byron_Pollo_Exit2_ST.txt не найдено совпадение года в данных из файла metadata.csv
Для файла Eduardo_Gimenez_Exit2_ST.txt не найдено совпадение года в данных из файла metadata.csv
Для файла Caleb_Herrmann_Exit2_ST.t